In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statsmodels.api as api
import time
import warnings

warnings.filterwarnings('ignore')

# Description Statistics

In [2]:
def get_desc(data):
    #['count mean std min max'.split()]
    desc = data.describe().T.applymap(lambda x:'{:.3f}'.format(x))
    desc['count'] = desc['count'].astype('float').astype('int').astype('object')
    desc.columns = ['观测值','均值','标准差','最小值','25%分位数','中位数','75%分位数','最大值']
    desc.index.name = '变量'
    return desc

In [3]:
rsj=pd.read_csv("D:/jupyterfile/empirical/数据/all_data/RSJ.csv",parse_dates=[1],index_col=[0,1])
rsj

ret_month       RSJ      beta        bm    coskew  \
code     date                                                            
1.0      2003-09-30  -0.113958  0.049737  1.287230  0.223160 -0.099491   
         2003-10-31   0.027849 -0.427933  1.272123  0.251398 -0.110652   
         2003-11-30   0.013878 -0.208668  1.289896  0.244198 -0.115375   
         2003-12-31   0.088847 -0.078250  1.337315  0.239453 -0.090127   
         2004-01-31   0.114744  0.018934  1.330531  0.243387 -0.090496   
...                        ...       ...       ...       ...       ...   
603999.0 2020-07-31   0.025908 -0.194310  1.318162  0.453179 -0.123373   
         2020-08-31  -0.152489  0.014451  1.255334  0.441201 -0.106651   
         2020-09-30  -0.032383  0.007407  1.180690  0.529022 -0.102166   
         2020-10-31   0.123759  0.061988  1.154601  0.546026 -0.122375   
         2020-11-30  -0.125051  0.127522  1.146227  0.485356 -0.104152   

                      idioVol     illiq    log_cap       Max       mom  
code     date                                                           
1.0      2003-09-30  0.008561  0.026030  23.604888  0.015919 -0.249819  
         2003-10-31  0.010569  0.034749  23.485741  0.025028 -0.280181  
         2003-11-30  0.011031  0.037089  23.514798  0.049438 -0.316147  
         2003-12-31  0.010719  0.026259  23.530192  0.034347 -0.187932  
         2004-01-31  0.013839  0.019322  23.616812  0.045909 -0.256170  
...                       ...       ...        ...       ...       ...  
603999.0 2020-07-31  0.013449  0.039779  22.063223  0.049322  0.122639  
         2020-08-31  0.013760  0.027672  22.090010  0.034860  0.185846  
         2020-09-30  0.009352  0.052888  21.926022  0.028317  0.211573  
         2020-10-31  0.007505  0.079706  21.894385  0.024180 -0.008526  
         2020-11-30  0.014892  0.067541  22.012168  0.055235 -0.114921  

[338730 rows x 10 columns]

In [4]:
get_desc(rsj)

,观测值,均值,标准差,最小值,25%分位数,中位数,75%分位数,最大值
变量,,,,,,,,
ret_month,338730,0.010,0.134,-0.616,-0.069,-0.003,0.074,2.499
RSJ,338730,0.021,0.199,-0.890,-0.114,0.023,0.161,0.791
beta,338730,1.146,0.243,0.140,0.998,1.145,1.293,2.373
bm,338730,0.464,0.298,0.000,0.251,0.393,0.599,2.520
coskew,338730,-0.013,0.064,-0.230,-0.057,-0.014,0.031,0.201
idioVol,338730,0.016,0.007,0.000,0.011,0.015,0.020,0.061
illiq,338730,0.129,0.275,0.000,0.018,0.041,0.103,4.976
log_cap,338730,22.288,1.000,19.133,21.596,22.181,22.875,26.264
Max,338730,0.049,0.024,-0.000,0.030,0.043,0.062,0.109


In [5]:
def get_corr_mat(data):
    x = []
    for i in range(data.shape[1]):
        y = []
        for j in range(i+1):
            r = st.pearsonr(data.iloc[:,i],data.iloc[:,j])   #获得两列之间的相关系数和p值
            y.append(r)
        x.append(y)
    x = pd.DataFrame(x,data.columns,data.columns)
    
    t = x.applymap(lambda x:x[0],'ignore').applymap(lambda x:'{:.2f}'.format(x))
    p = x.applymap(lambda x:x[1],'ignore')
    
    p_ = p.copy().astype('object')
    p_[p<=0.1] = '*'
    p_[p<=0.05] = '**'
    p_[p<=0.01] = '***'
    p_[p>0.01] = ''
    p = p_
    
    for i in range(data.shape[1]):
        for j in range(i):
            try:
                t.iloc[i,j] = t.iloc[i,j] + p.iloc[i,j]
            except:
                t.iloc[i,j] = ''
    t[t=='1.00'] = '1'
    t[t=='nan'] = ''
    return t

In [6]:
#内部分解测试
#a=rsj.iloc[:100]
#x = []
#for i in range(a.shape[1]):
#    y = []
#    for j in range(i+1):
#        r = st.pearsonr(a.iloc[:,i],a.iloc[:,j])   #获得两列之间的相关系数和p值
#        y.append(r)
#    x.append(y)
#x = pd.DataFrame(x,a.columns,a.columns)
#
#t = x.applymap(lambda x:x[0],'ignore').applymap(lambda x:'{:.2f}'.format(x))
#p = x.applymap(lambda x:x[1],'ignore')

In [7]:
def group(df):
    group_list = []
    q = [0,0.2,0.4,0.6,0.8,1]
    for i in range(len(q)-1):
        q1=q[i]
        q2=q[i+1]
        if q2==1:
            temp_group = df.apply(lambda x:(x>x.quantile(q1))&(x<=x.quantile(q2)),axis=1) * 1.0
        else:
            temp_group = df.apply(lambda x:(x>=x.quantile(q1))&(x<=x.quantile(q2)),axis=1) * 1.0
        temp_group = temp_group.replace(0,np.nan)
        group_list.append(temp_group)
    return group_list

def get_sort_desc_mat(factor,data_):
    data = data_.groupby(['date','code']).mean().unstack()
    group_list = group(data[factor])
    result = []
    for i in data_.columns:
        result.append([(data[i]*group_).mean(axis=1) for group_ in group_list]) #每个因子和被五等份的因子相乘，先求横向求平均值
    result = pd.concat([pd.concat(i,axis=1).mean() for i in result],axis=1).T #每个因子的5个数据横向合并且纵向求均值，将每个因子的结果合并
    result.index = data_.columns
    result[10] = result.iloc[:,-1] - result.iloc[:,0]
    result = result.applymap(lambda x:'{:.3f}'.format(x))
    if result.shape[1]<10: 
        result.columns = ['L','2','3','4','H','H-L']
    else:
        result.columns = ['L','2','3','4','5','6','7','8','9','H','H-L']
    result = result.astype('object')
    return result

In [33]:
get_sort_desc_mat("RSJ",rsj)

,L,2,3,4,H,H-L
ret_month,0.013,0.012,0.010,0.008,0.005,-0.008
RSJ,-0.202,-0.065,0.017,0.099,0.231,0.432
beta,1.134,1.156,1.154,1.138,1.080,-0.055
bm,0.511,0.475,0.449,0.423,0.370,-0.141
coskew,-0.036,-0.026,-0.016,-0.003,0.023,0.059
idioVol,0.014,0.015,0.016,0.017,0.019,0.005
illiq,0.198,0.203,0.190,0.170,0.131,-0.067
log_cap,21.952,21.999,22.083,22.199,22.459,0.506
Max,0.043,0.046,0.049,0.052,0.056,0.014
mom,-0.011,0.053,0.120,0.207,0.382,0.392


In [14]:
factors = 'Eco_mov1,Co_mov,Beta,Beta_down,Beta_up,Size,Bm,Mom,Str,Illiq,\
Illiq_std,Tover,Beta1,Beta2,Beta3,Beta4'.split(',')
data = pd.read_csv('../数据/all_data/all_data_del.csv',index_col=[0,1])[factors]

In [40]:
desc1 = get_desc(data)
# desc1.to_excel('../result/描述性统计1.xlsx')
desc1

,观测值,均值,标准差,最小值,25%分位数,中位数,75%分位数,最大值
变量,,,,,,,,
Eco_mov1,336523,2.754,5.018,-93.225,0.294,1.658,4.359,71.519
Co_mov,336523,2.063,1.156,-2.125,1.322,1.886,2.594,15.090
Beta,336523,1.127,0.249,0.147,0.980,1.132,1.281,2.373
Beta_down,336523,1.202,0.359,-0.456,0.994,1.217,1.417,3.206
Beta_up,336523,1.014,0.300,-0.394,0.830,1.005,1.191,2.624
Size,336523,22.325,0.990,19.238,21.641,22.216,22.906,26.338
Bm,336523,0.486,0.317,0.000,0.262,0.408,0.623,2.776
Mom,336523,0.102,0.544,-0.836,-0.229,-0.033,0.262,7.494
Str,336523,0.007,0.122,-0.478,-0.067,-0.002,0.070,1.154


In [47]:
desc2 = get_sort_desc_mat('Eco_mov1',data)
# desc2.to_excel('../result/描述性统计2.xlsx')
desc2

,L,2,3,4,H,H-L
Eco_mov1,-1.409,0.458,1.684,3.663,8.996,10.405
Co_mov,1.866,1.960,2.102,2.213,2.318,0.452
Beta,1.084,1.122,1.141,1.147,1.136,0.051
Beta_down,1.118,1.171,1.226,1.259,1.270,0.153
Beta_up,1.009,1.042,1.023,1.000,0.971,-0.039
Size,22.384,22.596,22.272,21.931,21.634,-0.749
Bm,0.444,0.472,0.475,0.471,0.448,0.005
Mom,0.166,0.147,0.132,0.116,0.117,-0.049
Str,0.010,0.008,0.007,0.008,0.009,-0.001
Illiq,0.150,0.103,0.135,0.182,0.257,0.107


In [8]:
t = get_corr_mat(data)
t.to_excel('../result/描述性统计3.xlsx',sheet_name='2')
t

,ECo_monent5,Co_monent1,Beta,Beta_down,Beta_up,Size,Bm,Mom,Str,Illiq,Illiq_std,Tover,Beta1,Beta2,Beta3,Beta4
ECo_monent5,1.00,,,,,,,,,,,,,,,
Co_monent1,0.13***,1.00,,,,,,,,,,,,,,
Beta,0.00,0.23***,1.00,,,,,,,,,,,,,
Beta_down,0.06***,0.24***,0.80***,1.00,,,,,,,,,,,,
Beta_up,-0.07***,0.11***,0.63***,0.31***,1.00,,,,,,,,,,,
Size,-0.23***,-0.19***,-0.09***,-0.20***,0.11***,1.00,,,,,,,,,,
Bm,-0.01***,-0.05***,-0.12***,-0.08***,-0.05***,0.07***,1.00,,,,,,,,,
Mom,-0.03***,-0.05***,-0.04***,-0.04***,-0.06***,0.20***,-0.15***,1.00,,,,,,,,
Str,-0.00,-0.02***,-0.03***,-0.02***,-0.04***,0.07***,-0.06***,0.31***,1.00,,,,,,,
Illiq,0.05***,0.03***,-0.03***,0.01***,-0.04***,-0.36***,0.01***,-0.15***,-0.11***,1.00,,,,,,
